In [42]:
# ! pip install kaggle
! pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 24.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [langchain]15 [langchain]text-splitters]


In [17]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('marawanxmamdouh/email-thread-summary-dataset', path='./data/', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/marawanxmamdouh/email-thread-summary-dataset


In [43]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For splitting large text into smaller chunks

In [61]:
email_details = pd.read_csv('data/CSV/email_thread_details.csv')
email_summaries = pd.read_csv('data/CSV/email_thread_summaries.csv')

In [62]:
email_details.head(1)

,thread_id,subject,timestamp,from,to,body
0,1,FW: Master Termination Log,2002-01-29 11:23:42,"Gossett, Jeffrey C. JGOSSET","['Giron', 'Darron C. Dgiron', 'Love', 'Phillip...",\n\n -----Original Message-----\nFrom: =09Ther...


In [63]:
email_details[email_details['thread_id'] == 10]

,thread_id,subject,timestamp,from,to,body
48,10,RE: NNG/Dynegy Due Diligence,2001-11-28 06:15:52,"Saunders, James JSAUNDE","['Hayslett', 'Rod Rhaysle', 'Howard', 'Kevin A...",yes...we can identify components via supportin...
49,10,RE: NNG/Dynegy Due Diligence,2001-11-28 06:31:53,"Hayslett, Rod RHAYSLE","['Saunders', 'James Jsaunde', 'Howard', 'Kevin...",Explain vs redo.\n--------------------------\n...
50,10,RE: NNG/Dynegy Due Diligence,2001-11-28 07:07:06,"Saunders, James JSAUNDE","['Hayslett', 'Rod Rhaysle', 'Howard', 'Kevin A...","redo = redo each of the fiancials i/s, b/s, c/..."
51,10,RE: NNG/Dynegy Due Diligence,2001-11-28 07:23:23,"Hayslett, Rod RHAYSLE","['Saunders', 'James Jsaunde', 'Howard', 'Kevin...","I meant to try and explain, not to redo. Only..."
52,10,RE: NNG/Dynegy Due Diligence,2001-11-28 07:33:11,"Saunders, James JSAUNDE","['Hayslett', 'Rod Rhaysle', 'Howard', 'Kevin A...",if no one has difficulty showing investments i...
53,10,RE: NNG/Dynegy Due Diligence,2001-11-28 07:54:38,"Hayslett, Rod RHAYSLE","['Saunders', 'James Jsaunde', 'Howard', 'Kevin...",Agreed. Don't make it harder than it needs to...
54,10,RE: NNG/Dynegy Due Diligence,2001-11-28 07:58:28,"Saunders, James JSAUNDE","['Hayslett', 'Rod Rhaysle', 'Howard', 'Kevin A...",thanks\n\n-----Original Message-----\nFrom: Ha...


In [64]:
email_summaries.head()

,thread_id,summary
0,1,The email thread discusses the Master Terminat...
1,2,A lunch meeting has been scheduled for May 5th...
2,3,Ben is updating a friend on his progress with ...
3,4,The recipient of the email thread initially ex...
4,5,The email thread discusses the long form confi...


In [65]:
email_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21684 entries, 0 to 21683
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   thread_id  21684 non-null  int64 
 1   subject    21684 non-null  object
 2   timestamp  21684 non-null  object
 3   from       21684 non-null  object
 4   to         21684 non-null  object
 5   body       21684 non-null  object
dtypes: int64(1), object(5)
memory usage: 1016.6+ KB


In [66]:
email_summaries

,thread_id,summary
0,1,The email thread discusses the Master Terminat...
1,2,A lunch meeting has been scheduled for May 5th...
2,3,Ben is updating a friend on his progress with ...
3,4,The recipient of the email thread initially ex...
4,5,The email thread discusses the long form confi...
...,...,...
4162,4163,Peter Thompson has sent a memo to Kay Mann and...
4163,4164,The email thread revolves around the sharing a...
4164,4165,Susan asks Emily about her plans for the weeke...
4165,4166,Several employees will be on vacation during d...


In [67]:
# merge the body text with summaries based on thread id

email_summaries = email_summaries.merge(email_details[['thread_id', 'body']], on='thread_id', how='left')

In [69]:
email_summaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21684 entries, 0 to 21683
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   thread_id  21684 non-null  int64 
 1   summary    21684 non-null  object
 2   body       21684 non-null  object
dtypes: int64(1), object(2)
memory usage: 508.3+ KB


In [87]:
print(email_summaries['body'][28])

No.


From: Reagan Rorschach/ENRON@enronXgate on 04/30/2001 07:56 AM
To: Kay Mann/Corp/Enron@Enron
cc:  

Subject: RE: long form confirm/MDEA

Kay, have these issues been incorporated into the ILA?

 -----Original Message-----
From:  Mann, Kay  
Sent: Sunday, April 29, 2001 11:51 AM
To: Sacks, Edward
Cc: Rorschach, Reagan; kay.mann@worldnet.att.net
Subject: RE: long form confirm/MDEA

Instinctively it seems that the agent for concept would be stronger.  I'll 
check into it for the  longer term deal.  Obviously, it is a Mississippi law 
issue, so I'll have to get an opinion.

Kay


From: Edward Sacks/ENRON@enronXgate on 04/29/2001 10:46 AM
To: Reagan Rorschach/ENRON@enronXgate, Kay Mann/Corp/Enron@Enron, 
kay.mann@worldnet.att.net@SMTP@enronXgate
cc:  

Subject: RE: long form confirm/MDEA

The following are my comments:

  - I assume that Settlements is fine with invoicing no later than the 7 th 
of each mth.
  - Section M:  Should we go this route (covenant of financial support) or 
wo

In [86]:
# create a text splitter instance
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # maximum size of each chunk
    chunk_overlap=50  # overlap between chunks to maintain context
)